# Algorithm 23: Smooth LDDT (AlphaFold3)

Differentiable version of LDDT score used as a training loss.

## Source Code Location
- **File**: `AF3-Ref-src/alphafold3-official/src/alphafold3/model/network/losses.py`

## Overview

### LDDT (Local Distance Difference Test)

LDDT measures local structure quality by comparing distances in predicted and true structures:

```
LDDT = fraction of distances within cutoffs [0.5, 1, 2, 4] Angstroms
```

### Smooth LDDT
- Uses sigmoid instead of hard threshold
- Differentiable for gradient-based training
- Provides direct structural supervision

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
def compute_distances(coords):
    """
    Compute pairwise distances.
    
    Args:
        coords: [N, 3] coordinates
    
    Returns:
        distances: [N, N] pairwise distances
    """
    diff = coords[:, None, :] - coords[None, :, :]
    return np.sqrt(np.sum(diff ** 2, axis=-1))

In [ ]:
def lddt_score(pred_coords, true_coords, cutoff=15.0):
    """
    Compute standard (hard) LDDT score.
    
    Args:
        pred_coords: Predicted coordinates [N, 3]
        true_coords: Ground truth coordinates [N, 3]
        cutoff: Distance cutoff for considering pairs
    
    Returns:
        Per-residue LDDT scores [N]
    """
    thresholds = [0.5, 1.0, 2.0, 4.0]
    
    pred_dists = compute_distances(pred_coords)
    true_dists = compute_distances(true_coords)
    
    # Only consider pairs within cutoff in true structure
    mask = (true_dists < cutoff) & (true_dists > 0)  # Exclude self-pairs
    
    # Distance differences
    dist_diff = np.abs(pred_dists - true_dists)
    
    # Count pairs within each threshold
    scores = []
    for thresh in thresholds:
        within = (dist_diff < thresh) & mask
        score_per_residue = np.sum(within, axis=1) / np.maximum(np.sum(mask, axis=1), 1)
        scores.append(score_per_residue)
    
    # Average over thresholds
    lddt = np.mean(scores, axis=0)
    
    return lddt

In [ ]:
def smooth_lddt_score(pred_coords, true_coords, cutoff=15.0, temperature=0.5):
    """
    Compute smooth (differentiable) LDDT score.
    
    Uses sigmoid instead of hard threshold.
    
    Args:
        pred_coords: Predicted coordinates [N, 3]
        true_coords: Ground truth coordinates [N, 3]
        cutoff: Distance cutoff for considering pairs
        temperature: Sigmoid temperature (lower = sharper)
    
    Returns:
        Per-residue smooth LDDT scores [N]
    """
    thresholds = [0.5, 1.0, 2.0, 4.0]
    
    pred_dists = compute_distances(pred_coords)
    true_dists = compute_distances(true_coords)
    
    # Soft mask using sigmoid
    mask = 1 / (1 + np.exp((true_dists - cutoff) / temperature))
    mask = mask * (1 - np.eye(len(true_coords)))  # Exclude self-pairs
    
    # Distance differences
    dist_diff = np.abs(pred_dists - true_dists)
    
    # Smooth threshold using sigmoid
    scores = []
    for thresh in thresholds:
        # Sigmoid: 1 when dist_diff << thresh, 0 when dist_diff >> thresh
        within = 1 / (1 + np.exp((dist_diff - thresh) / (temperature * thresh)))
        score_per_residue = np.sum(within * mask, axis=1) / np.maximum(np.sum(mask, axis=1), 1e-8)
        scores.append(score_per_residue)
    
    # Average over thresholds
    lddt = np.mean(scores, axis=0)
    
    return lddt

In [ ]:
def smooth_lddt_loss(pred_coords, true_coords, cutoff=15.0):
    """
    Smooth LDDT loss (1 - LDDT).
    
    Returns scalar loss for training.
    """
    lddt = smooth_lddt_score(pred_coords, true_coords, cutoff)
    loss = 1 - np.mean(lddt)
    return loss

In [ ]:
# Test
print("Test: Smooth LDDT")
print("="*60)

N = 32

# Ground truth structure
true_coords = np.random.randn(N, 3) * 5

# Test with different prediction qualities
noise_levels = [0.0, 0.5, 1.0, 2.0, 5.0]

print(f"{'Noise':<10} {'Hard LDDT':<15} {'Smooth LDDT':<15} {'Loss':<10}")
print("-" * 50)

for noise in noise_levels:
    pred_coords = true_coords + np.random.randn(N, 3) * noise
    
    hard_lddt = np.mean(lddt_score(pred_coords, true_coords))
    soft_lddt = np.mean(smooth_lddt_score(pred_coords, true_coords))
    loss = smooth_lddt_loss(pred_coords, true_coords)
    
    print(f"{noise:<10.1f} {hard_lddt:<15.4f} {soft_lddt:<15.4f} {loss:<10.4f}")

In [ ]:
# Test: Gradient approximation
print("\nTest: Gradient Sensitivity")
print("="*60)

pred_coords = true_coords + np.random.randn(N, 3) * 1.0

# Numerical gradient check
eps = 0.01
base_loss = smooth_lddt_loss(pred_coords, true_coords)

# Perturb one coordinate
pred_perturbed = pred_coords.copy()
pred_perturbed[0, 0] += eps
perturbed_loss = smooth_lddt_loss(pred_perturbed, true_coords)

numerical_grad = (perturbed_loss - base_loss) / eps

print(f"Base loss: {base_loss:.6f}")
print(f"Perturbed loss: {perturbed_loss:.6f}")
print(f"Numerical gradient (approx): {numerical_grad:.6f}")
print(f"Gradient is non-zero: {abs(numerical_grad) > 1e-8}")

## Key Insights

1. **Differentiable**: Smooth sigmoid enables gradient flow
2. **Local Focus**: Only considers pairs within 15A cutoff
3. **Multi-threshold**: Averages over 4 distance thresholds
4. **Direct Supervision**: Provides structural loss beyond FAPE/diffusion